In [32]:
import requests
import pandas as pd 
from datetime import timedelta
import io

StatementMeta(, 8282ec02-5d7e-459e-9941-9ee7b84859fa, 34, Finished, Available, Finished)

In [33]:
# Base URL for Github raw CSV file

base_url = "https://raw.githubusercontent.com/mikailaltundas/datasets-for-training/main/wind-power-dataset/"
# Path to the lake house Broonze
broze_table_path = "abfss://WINDPOWERGENERATION@onelake.dfs.fabric.microsoft.com/LH_BRONZE.Lakehouse/Tables/wind_power_production"



StatementMeta(, 8282ec02-5d7e-459e-9941-9ee7b84859fa, 35, Finished, Available, Finished)

In [34]:
# Load the existing wind_power_production table into a Pandas DataFrame 
df_spark = spark.read.format("delta").load(broze_table_path)
df_pandas = df_spark.toPandas()

StatementMeta(, 8282ec02-5d7e-459e-9941-9ee7b84859fa, 36, Finished, Available, Finished)

In [35]:
# Get the most recent date and the next day

most_recent_date = pd.to_datetime(df_pandas["date"], format = '%Y%m%d').max()
next_date = (most_recent_date + timedelta(days = 1)).strftime('%Y%m%d')

# From the URL download the next day's CSV file 
next_file_url = f"{base_url}{next_date}_wind_power_production.csv"
file_content = requests.get(next_file_url).content

# Load the CSV into the a pandas DataFrame 
df_pandas_new = pd.read_csv(io.StringIO(file_content.decode('utf-8')))
# Convert the date column to the datetime 
df_pandas_new['date'] = pd.to_datetime(df_pandas_new['date'])
# Convert pandas dataFrame to Spark DataFrame
df_spark_new = spark.createDataFrame(df_pandas_new, schema=df_spark.schema) 

StatementMeta(, 8282ec02-5d7e-459e-9941-9ee7b84859fa, 37, Finished, Available, Finished)

In [36]:
# Append the ne data to the existing wind_power_production table to the Bronze Lakehouse
df_spark_new.write.format("delta").mode("append").save(broze_table_path)

StatementMeta(, 8282ec02-5d7e-459e-9941-9ee7b84859fa, 38, Finished, Available, Finished)